In [ ]:
%config InlineBackend.figure_format='png'
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import colorcet as cc
import cartopy.crs as crs
from netCDF4 import Dataset as ds
import xarray as xr

from datetime import datetime, timedelta
from cftime import date2num, num2pydate
import pickle

In [ ]:
# no data for stations # 2, 9, 10
# significant portion of data missing for stations # 1
# There are also portions of data missing for several other stations but mainly in the earlier period.
rela = ds("/ccs/home/h1x/scratchp/wrfhydro_cacti/relampago/RELAMPAGO_Site_Data_20180501_20190501.nc")
lonp = rela['lon'][:]
latp = rela['lat'][:]

In [ ]:
def find_ind(lati, loni, latp, lonp, dom):
    id1 = np.empty(15, dtype=np.int64) 
    id2 = np.empty(15, dtype=np.int64) 
    for i in range(15):
        a, b = np.where((lati[1:,:] > latp[i]) & (lati[:-1,:] <= latp[i]) & (loni[:,1:] > lonp[i]) & (loni[:, :-1] <= lonp[i]))
        if len(a)*len(b) == 0:
            print(f"Station {i+1} at {lonp[i]}E, {latp[i]}N is not in the domain {dom}.")
            id1[i] = -9999
            id2[i] = -9999
        else:
            id1[i] = a[0]
            id2[i] = b[0]
    return id1, id2

era5land_d1 = ds("/ccs/home/h1x/scratchp/wrfhydro_cacti/era5-land/regridded/d01/swvl1_201901250000.nc")
era5land_d2 = ds("/ccs/home/h1x/scratchp/wrfhydro_cacti/era5-land/regridded/d02/swvl1_201901250000.nc")
wo_d2 = ds(
    "/ccs/home/h1x/scratchp/wrfhydro_cacti/wrfhydro2wrfinput/original"
    "/20190125/era5eda_en01"
    "/run20190125_thom_k150ptop11_dzbot25_dzsts1.05_dzstu1.05/wrfinput_d02"
)
wo_d1 = ds(
    "/ccs/home/h1x/scratchp/wrfhydro_cacti/wrfhydro2wrfinput/original"
    "/20190125/era5eda_en01"
    "/run20190125_thom_k150ptop11_dzbot25_dzsts1.05_dzstu1.05/wrfinput_d01"
)
lati_d1 = wo_d1['XLAT_V'][0,:,:]
loni_d1 = wo_d1['XLONG_U'][0,:,:]
lati_d2 = wo_d2['XLAT_V'][0,:,:]
loni_d2 = wo_d2['XLONG_U'][0,:,:]
id1_d1, id2_d1 = find_ind(lati_d1, loni_d1, latp, lonp, 1)
id1_d2, id2_d2 = find_ind(lati_d2, loni_d2, latp, lonp, 2)

In [ ]:
def plot_sml1(dom, forcing, date, lonp, latp):
    wo = ds(
        f"/ccs/home/h1x/scratchp/wrfhydro_cacti/wrfhydro2wrfinput/original/{date[:8]}/{forcing}/run20190125_thom_k150ptop11_dzbot25_dzsts1.05_dzstu1.05/wrfinput_{dom}"
    )
    wm = ds(
        f"/ccs/home/h1x/scratchp/wrfhydro_cacti/wrfhydro2wrfinput/modified/{date[:8]}/{forcing}/run20190125_thom_k150ptop11_dzbot25_dzsts1.05_dzstu1.05/wrfinput_{dom}"
    )
    era5land = ds(
        f"/ccs/home/h1x/scratchp/wrfhydro_cacti/era5-land/regridded/{dom}/swvl1_{date}.nc"
    )

    titles = [f"WRF-{forcing}", "WRF-Hydro", "ERA5-land"]
    vars = [
        wo["SMOIS"][0, 0, :, :],
        wm["SMOIS"][0, 0, :, :],
        era5land["swvl1"][0, :, :],
    ]
    lon = wo["XLONG"][0, :, :]
    lat = wo["XLAT"][0, :, :]

    sns.set()
    sns.set_context("poster")
    sns.set_style("ticks")
    # levs = None
    levs = np.arange(0.0, 0.65, 0.08)
    fig = plt.figure(figsize=(25, 9))
    fig.suptitle(f"Topmost layer Soil Moisture (m**3/m**3) - {dom} - {date}")
    axes = fig.subplots(1, 3, subplot_kw={"projection": crs.PlateCarree()})
    for ax, var, title in zip(axes, vars, titles):
        ax.contour(lon, lat, var, levels=levs, linewidths=1.5, colors="black")
        ct = ax.contourf(
            lon, lat, var, levels=levs, extend="neither", cmap=cc.cm.coolwarm
        )
        ax.set_xlim(lon.min(), lon.max())
        ax.set_ylim(lat.min(), lat.max())
        gl = ax.gridlines(
            color="black",
            linestyle="dotted",
            draw_labels=True,
            x_inline=False,
            y_inline=False,
        )
        gl.xlocator = ticker.MultipleLocator(3)
        gl.ylocator = ticker.MultipleLocator(2)
        gl.right_labels = False
        gl.top_labels = False
        ax.set_title(title)
        for i in range(15):
            ax.text(
                lonp[i],
                latp[i],
                f"{i+1}",
                fontsize=15,
                fontweight="bold",
                ha="center",
                va="center",
                color="green",
            )
    plt.colorbar(ct, ax=axes, fraction=0.05, orientation="horizontal")
    # plt.tight_layout()
    fig.savefig(f"sml1.{dom}.{forcing}.{date}.png")
    plt.show()

In [ ]:
plot_sml1('d01', 'era5eda_en01', '201901250000', lonp, latp)
plot_sml1('d02', 'era5eda_en01', '201901250000', lonp, latp)

In [ ]:
start_time = datetime.strptime("201808010100", "%Y%m%d%H%M")
end_time = datetime.strptime("201903210000", "%Y%m%d%H%M")
dt = timedelta(seconds=3600)
nt = int((end_time - start_time)/dt) + 1

In [ ]:
def extract_stns(dom, start_time, nt, dt, id1, id2):
    sme = np.zeros((15, nt))
    smh = np.zeros((15, nt))
    for t in range(nt):
        date = (start_time + t * dt).strftime("%Y%m%d%H%M")
        print(date)
        era = ds(
            f"/ccs/home/h1x/scratchp/wrfhydro_cacti/era5-land/regridded/{dom}/swvl1_{date}.nc"
        )
        wrfhydro = ds(
            f"/ccs/home/h1x/scratchp/wrfhydro_cacti/wrfhydro_run/run01_{dom}/out/{date}.LDASOUT_DOMAIN{dom[-1]}"
        )
        sm_era = era["swvl1"][0, :, :]
        sm_wrfhydro = wrfhydro["SOIL_M"][0, :, 0, :]
        for s in range(15):
            if id1[s] != -9999:
                sme[s, t] = sm_era[id1[s], id2[s]]
                smh[s, t] = sm_wrfhydro[id1[s], id2[s]]
        era.close()
        wrfhydro.close()
    with open(f"sme_{dom}.pkl", "wb") as f:
        pickle.dump(sme, f, protocol=-1)
    with open(f"smh_{dom}.pkl", "wb") as f:
        pickle.dump(smh, f, protocol=-1)
    return
# extract_stns('d01', start_time, nt, dt, id1_d1, id2_d1)
# extract_stns('d02', start_time, nt, dt, id1_d2, id2_d2)

In [ ]:
with open('sme_d01.pkl', 'rb') as f:
    sme_d01 = pickle.load(f)
with open('sme_d02.pkl', 'rb') as f:
    sme_d02 = pickle.load(f)
with open('smh_d01.pkl', 'rb') as f:
    smh_d01 = pickle.load(f)
with open('smh_d02.pkl', 'rb') as f:
    smh_d02 = pickle.load(f)

In [ ]:
tm = [start_time+i*dt for i in range(nt)]
trn = rela['time'][:]
units = rela['time'].units
trn_m = date2num(tm, units=units)
ind_m  = np.in1d(trn, trn_m).nonzero()[0]
tr = [num2pydate(t, units=units) for t in trn]

In [ ]:
def plot_station(i, smh_d01, sme_d01, smh_d02, sme_d02, tm, rela, tr, ind_m):
    sns.set()
    sns.set_context('poster')
    sns.set_style('ticks')
    fig = plt.figure(figsize=(18,6))
    rela_m = rela['Soil_moist_5cm'][i-1, ind_m]
    me_h = (smh_d01[i-1,:] - rela_m).mean()
    me_e = (sme_d01[i-1,:] - rela_m).mean()
    rmse_h = np.sqrt(((smh_d01[i-1,:] - rela_m)**2).mean())
    rmse_e = np.sqrt(((sme_d01[i-1,:] - rela_m)**2).mean())
    plt.plot(tm, smh_d01[i-1,:], color='red', label=f'WRF-HYDRO D01, M.Err.={me_h:.6f}, RMSE={rmse_h:.6f}')
    # plt.plot(tm, smh_d02[i-1,:], color='red', linestyle='--', label='WRF-HYDRO D02')
    plt.plot(tm, sme_d01[i-1,:], color='blue', label=f'ERA5-land D01, M.Err.={me_e:.6f}, RMSE={rmse_e:.6f}')
    # plt.plot(tm, sme_d02[i-1,:], color='blue', linestyle='--', label='ERA5-land D02')
    plt.plot(tr, rela['Soil_moist_5cm'][i-1,:], color='black', label='RELAMPAGO')
    plt.legend()
    plt.xlim(tm[0], tm[-1])
    plt.ylim(0, 0.6)
    plt.title(f'Station #{i} Top Layer Soil Moisture (m**3/m**3)')
    fig.savefig(f'sml1.stn{i}.png')
    plt.show()

In [ ]:
for i in range(15):
    plot_station(i+1, smh_d01, sme_d01, smh_d02, sme_d02, tm, rela, tr, ind_m)

In [ ]:
# east = np.asarray([3, 6, 13, 5]) - 1
east1 = np.asarray([3, 6, 13, 5]) - 1
east2 = np.asarray([3, 13, 5]) - 1
west = np.asarray([4, 8, 12, 15]) - 1
g_smh_d01 = smh_d01[east1,:].mean(axis=0) - smh_d01[west,:].mean(axis=0)
g_smh_d02 = smh_d02[east2,:].mean(axis=0) - smh_d02[west,:].mean(axis=0)
g_sme_d01 = sme_d01[east1,:].mean(axis=0) - sme_d01[west,:].mean(axis=0)
g_sme_d02 = sme_d02[east2,:].mean(axis=0) - sme_d02[west,:].mean(axis=0)
g_rela_d01 = rela['Soil_moist_5cm'][east1,:].mean(axis=0) - rela['Soil_moist_5cm'][west,:].mean(axis=0)
g_rela_d02 = rela['Soil_moist_5cm'][east2,:].mean(axis=0) - rela['Soil_moist_5cm'][west,:].mean(axis=0)
g_rela_d01_m = g_rela_d01[ind_m]
g_rela_d02_m = g_rela_d02[ind_m]

In [ ]:
print(
    g_smh_d01.mean(),
    g_smh_d02.mean(),
    g_sme_d01.mean(),
    g_sme_d02.mean(),
    g_rela_d01_m.mean(),
    g_rela_d02_m.mean(),
)



In [ ]:
me_g_smh_d01 = g_smh_d01.mean() - g_rela_d01_m.mean()
me_g_sme_d01 = g_sme_d02.mean() - g_rela_d02_m.mean()

In [ ]:
me_g_smh_d01_alt = g_smh_d01[1464:].mean() - g_rela_d01_m[1464:].mean()
me_g_sme_d01_alt = g_sme_d02[1464:].mean() - g_rela_d02_m[1464:].mean()

In [ ]:
rmse_g_smh_d01 = np.sqrt(((g_smh_d01 - g_rela_d01_m) ** 2).mean())
rmse_g_sme_d01 = np.sqrt(((g_sme_d01 - g_rela_d01_m) ** 2).mean())

In [ ]:
rmse_g_smh_d01_alt = np.sqrt(((g_smh_d01[1464:] - g_rela_d01_m[1464:]) ** 2).mean())
rmse_g_sme_d01_alt = np.sqrt(((g_sme_d01[1464:] - g_rela_d01_m[1464:]) ** 2).mean())

In [ ]:
print(
    g_smh_d01[1464:].mean(),
    g_smh_d02[1464:].mean(),
    g_sme_d01[1464:].mean(),
    g_sme_d02[1464:].mean(),
    g_rela_d01_m[1464:].mean(),
    g_rela_d02_m[1464:].mean(),
)



In [ ]:
print(
    np.sqrt(((g_smh_d01 - g_rela_d01_m) ** 2).mean()),
    np.sqrt(((g_smh_d02 - g_rela_d02_m) ** 2).mean()),
    np.sqrt(((g_sme_d01 - g_rela_d01_m) ** 2).mean()),
    np.sqrt(((g_sme_d02 - g_rela_d02_m) ** 2).mean()),
)



In [ ]:
print(
    np.sqrt(((g_smh_d01[1464:] - g_rela_d01_m[1464:]) ** 2).mean()),
    np.sqrt(((g_smh_d02[1464:] - g_rela_d02_m[1464:]) ** 2).mean()),
    np.sqrt(((g_sme_d01[1464:] - g_rela_d01_m[1464:]) ** 2).mean()),
    np.sqrt(((g_sme_d02[1464:] - g_rela_d02_m[1464:]) ** 2).mean()),
)



In [ ]:
sns.set()
sns.set_context('poster')
sns.set_style('ticks')
fig = plt.figure(figsize=(18,6))
plt.plot(tm, g_sme_d01, color='blue', label=f'ERA5-land, M.Err.={me_g_sme_d01:.6f}({me_g_sme_d01_alt:.6f}), RMSE={rmse_g_sme_d01:.6f}({rmse_g_sme_d01_alt:.6f})')
plt.plot(tm, g_smh_d01, color='red', label=f'WRF-HYDRO, M.Err.={me_g_smh_d01:.6f}({me_g_smh_d01_alt:.6f}), RMSE={rmse_g_smh_d01:.6f}({rmse_g_smh_d01_alt:.6f})')
# plt.plot(tm, g_smh_d02, color='red', linestyle='--', label='WRF-HYDRO D02')
# plt.plot(tm, g_sme_d02, color='blue', linestyle='--', label='ERA5-land D02')
plt.plot(tr, g_rela, color='black', label='RELAMPAGO')
plt.legend()
plt.xlim(tm[0], tm[-1])
# plt.ylim(0, 0.6)
plt.title(f'East-West Top Layer Soil Moisture Difference (m**3/m**3) d01')
fig.savefig(f'sml1.d01.e-w.png')
plt.show()